In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from keras.models import Model
from keras.layers import Flatten
import tensorflow as tf
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Hàm chuyển đổi một tệp JSON thành định dạng đầu ra phù hợp
def process_json(json_data, folder_name, dataset_path):
    # Trích xuất thông tin từ tệp JSON
    image_path = os.path.join(dataset_path, folder_name, json_data["imagePath"])
    annotations = []

    for shape in json_data["shapes"]:
        points = shape["points"]
        x_min = min(points, key=lambda x: x[0])[0]
        x_max = max(points, key=lambda x: x[0])[0]
        y_min = min(points, key=lambda x: x[1])[1]
        y_max = max(points, key=lambda x: x[1])[1]

        width = x_max - x_min
        height = y_max - y_min

        annotations.append({
            "Label": shape["label"],
            "Bbox": [x_min, y_min, width, height]
        })

    return {
        "image_path": image_path,
        "annotations": annotations
    }

dataset_path = "/content/drive/MyDrive/Colab Notebooks/Local/Data"
# Tạo danh sách để lưu trữ dữ liệu định dạng đầu ra
formatted_data = []

for folder_name in os.listdir(dataset_path):
    json_folder = os.path.join(dataset_path, folder_name)

    # Xử lý dữ liệu từ các tệp JSON trong thư mục
    for filename in os.listdir(json_folder):
        if filename.endswith('.json'):
            file_path = os.path.join(json_folder, filename)
            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)
                formatted_data.append(process_json(json_data, folder_name, dataset_path))

from sklearn.model_selection import train_test_split

# Dữ liệu đã được chuẩn hóa và lưu trong biến formatted_data

# Tách dữ liệu thành tập huấn luyện (60%), tập xác thực (20%) và tập kiểm tra (20%)
train_data, temp_data = train_test_split(formatted_data, test_size=0.4, random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# In số lượng mẫu trong mỗi tập
print(f"Số lượng mẫu trong tập huấn luyện: {len(train_data)}")
print(f"Số lượng mẫu trong tập xác thực: {len(validation_data)}")
print(f"Số lượng mẫu trong tập kiểm tra: {len(test_data)}")

# Kích thước đầu vào mong muốn của mô hình
input_shape = (224, 224, 3)

# Chuyển đổi dữ liệu hình ảnh và nhãn thành NumPy arrays và điều chỉnh kích thước ảnh
def load_and_resize_images(data):
    images = []
    bboxes = []
    labels = []

    for item in data:
        image = cv2.imread(item['image_path'])
        image = cv2.resize(image, (input_shape[0], input_shape[1]))

        annotation = item["annotations"]
        bbox_list = []
        label_list = []
        image_list = []

        for ann in annotation:
            # Thay đổi nhãn "Airplane" thành 1 và "Helicopter" thành 0
            label = 1 if ann['Label'] == 'Airplane' else 0
            bbox_list.append(ann['Bbox'])
            label_list.append(label)
            image_list.append(image)

        images.extend(image_list)
        bboxes.extend(bbox_list)
        labels.extend(label_list)  # Sử dụng extend() để thêm các giá trị của label_list vào labels

    images = np.array(images)
    bboxes = np.array(bboxes)
    labels = np.array(labels)

    return images, [bboxes, labels]

# Chia dữ liệu thành các biến
# Chia dữ liệu thành các biến
x_train, [y_train_bbox, y_train_labels] = load_and_resize_images(train_data)
x_validation, [y_validation_bbox, y_validation_labels] = load_and_resize_images(validation_data)
x_test, [y_test_bbox, y_test_labels] = load_and_resize_images(test_data)


Số lượng mẫu trong tập huấn luyện: 300
Số lượng mẫu trong tập xác thực: 100
Số lượng mẫu trong tập kiểm tra: 100


In [6]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

# Xây dựng mô hình
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))

# Đầu ra cho bounding box (4 giá trị: x, y, width, height)
bbox_output = Dense(4, activation='linear', name='BBox')(model.layers[-1].output)

# Đầu ra cho label (1 giá trị xác suất)
label_output = Dense(1, activation='sigmoid', name='Label')(model.layers[-1].output)

# Tạo mô hình
model = Model(inputs=model.inputs, outputs=[bbox_output, label_output])

# Kết hợp hàm mất mát
model.compile(optimizer='adam',
              loss={'Label': 'binary_crossentropy', 'BBox': 'mean_squared_error'},
              metrics={"Label":'accuracy'})

# Kiểm tra mô hình
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 conv2d_input (InputLayer)   [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 222, 222, 32)         896       ['conv2d_input[0][0]']        
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 109, 109, 64)         18496     ['max_pooling2d[0][0]']   

In [11]:
# Huấn luyện mô hình với nhãn đã được mã hóa
history = model.fit(
    x_train,
    {'BBox': y_train_bbox, 'Label': y_train_labels},
    validation_data=(x_validation, {'BBox': y_validation_bbox, 'Label': y_validation_labels}),
    epochs=20,
    batch_size=10,
    verbose=1
)


Epoch 1/20
31/31 [==============================] - 34s 1s/step - loss: 3045.6929 - BBox_loss: 3043.0825 - Label_loss: 2.6111 - Label_accuracy: 0.9103 - val_loss: 278217.8438 - val_BBox_loss: 278205.0938 - val_Label_loss: 12.7366 - val_Label_accuracy: 0.7100
Epoch 2/20
31/31 [==============================] - 34s 1s/step - loss: 6033.1216 - BBox_loss: 6029.9053 - Label_loss: 3.2159 - Label_accuracy: 0.8937 - val_loss: 286477.1562 - val_BBox_loss: 286466.6875 - val_Label_loss: 10.5030 - val_Label_accuracy: 0.6800
Epoch 3/20
31/31 [==============================] - 38s 1s/step - loss: 16187.1416 - BBox_loss: 16183.5781 - Label_loss: 3.5641 - Label_accuracy: 0.8970 - val_loss: 250096.9219 - val_BBox_loss: 250079.2188 - val_Label_loss: 17.6970 - val_Label_accuracy: 0.7000
Epoch 4/20
31/31 [==============================] - 34s 1s/step - loss: 29093.1621 - BBox_loss: 29088.0664 - Label_loss: 5.0955 - Label_accuracy: 0.8771 - val_loss: 317524.5938 - val_BBox_loss: 317514.8125 - val_Label_los

In [12]:
# Lưu model vào đường dẫn được chỉ định
model.save('Localization_Airplane_or_Helicopter.keras')

In [13]:
# Đánh giá mô hình trên tập kiểm tra
test_scores = model.evaluate(
    x_test,
    {'BBox': np.array(y_test_bbox), 'Label': np.array(y_test_labels)},
    verbose=1
)

# In kết quả đánh giá
print("Test Losses:")
print("Bounding Box Loss:", test_scores[1])
print("Label Loss:", test_scores[2])
print("Accuracy:", test_scores[3])

4/4 [==============================] - 3s 636ms/step - loss: 383727.3750 - BBox_loss: 383721.6250 - Label_loss: 5.7643 - Label_accuracy: 0.8173
Test Losses:
Bounding Box Loss: 383721.625
Label Loss: 5.764334678649902
Accuracy: 0.817307710647583
